In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0     89      0  0:00:17  0:00:17 --:--:--   355
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9 MB 1.4 MB/s 
     |████████████████████████████████| 1.0 MB 36.7 MB/s 
+ set +ex
Installation succeeded!


In [3]:
from __future__ import division

import numpy as np
import six


def calc_semantic_segmentation_confusion(pred_labels, gt_labels):
    pred_labels = iter(pred_labels)
    gt_labels = iter(gt_labels)

    n_class = 0
    confusion = np.zeros((n_class, n_class), dtype=np.int64)
    for pred_label, gt_label in six.moves.zip(pred_labels, gt_labels):
        if pred_label.ndim != 2 or gt_label.ndim != 2:
            raise ValueError('ndim of labels should be two.')
        if pred_label.shape != gt_label.shape:
            raise ValueError('Shape of ground truth and prediction should'
                             ' be same.')
        pred_label = pred_label.flatten()
        gt_label = gt_label.flatten()

        # Dynamically expand the confusion matrix if necessary.
        lb_max = np.max((pred_label, gt_label))
        if lb_max >= n_class:
            expanded_confusion = np.zeros(
                (lb_max + 1, lb_max + 1), dtype=np.int64)
            expanded_confusion[0:n_class, 0:n_class] = confusion

            n_class = lb_max + 1
            confusion = expanded_confusion

        # Count statistics from valid pixels.
        mask = gt_label >= 0
        confusion += np.bincount(
            n_class * gt_label[mask].astype(int) +
            pred_label[mask], minlength=n_class**2).reshape((n_class, n_class))

    for iter_ in (pred_labels, gt_labels):
        # This code assumes any iterator does not contain None as its items.
        if next(iter_, None) is not None:
            raise ValueError('Length of input iterables need to be same')
    return confusion


def calc_semantic_segmentation_iou(confusion):
    iou_denominator = (confusion.sum(axis=1) + confusion.sum(axis=0) -
                       np.diag(confusion))
    iou = np.diag(confusion) / iou_denominator
    return iou


def eval_semantic_segmentation(pred_labels, gt_labels):
    confusion = calc_semantic_segmentation_confusion(
        pred_labels, gt_labels)
    iou = calc_semantic_segmentation_iou(confusion)
    pixel_accuracy = np.diag(confusion).sum() / confusion.sum()
    class_accuracy = np.diag(confusion) / np.sum(confusion, axis=1)

    return {'iou': iou, 'miou': np.nanmean(iou),
            'pixel_accuracy': pixel_accuracy,
            'class_accuracy': class_accuracy,
            'mean_class_accuracy': np.nanmean(class_accuracy)}

In [4]:
import chainer
import chainer.functions as F
import chainer.links as L


class UNet(chainer.Chain):

    def __init__(self, class_num=2, ignore_label=255):

        self.__class_num = class_num
        self.__ignore_label = ignore_label

        super(UNet, self).__init__(
            c0=L.Convolution2D(3, 32, 3, 1, 1),
            c1=L.Convolution2D(32, 64, 4, 2, 1),
            c2=L.Convolution2D(64, 64, 3, 1, 1),
            c3=L.Convolution2D(64, 128, 4, 2, 1),
            c4=L.Convolution2D(128, 128, 3, 1, 1),
            c5=L.Convolution2D(128, 256, 4, 2, 1),
            c6=L.Convolution2D(256, 256, 3, 1, 1),
            c7=L.Convolution2D(256, 512, 4, 2, 1),
            c8=L.Convolution2D(512, 512, 3, 1, 1),

            dc8=L.Deconvolution2D(1024, 512, 4, 2, 1),
            dc7=L.Convolution2D(512, 256, 3, 1, 1),
            dc6=L.Deconvolution2D(512, 256, 4, 2, 1),
            dc5=L.Convolution2D(256, 128, 3, 1, 1),
            dc4=L.Deconvolution2D(256, 128, 4, 2, 1),
            dc3=L.Convolution2D(128, 64, 3, 1, 1),
            dc2=L.Deconvolution2D(128, 64, 4, 2, 1),
            dc1=L.Convolution2D(64, 32, 3, 1, 1),
            dc0=L.Convolution2D(64, class_num, 3, 1, 1),

            bnc0=L.BatchNormalization(32),
            bnc1=L.BatchNormalization(64),
            bnc2=L.BatchNormalization(64),
            bnc3=L.BatchNormalization(128),
            bnc4=L.BatchNormalization(128),
            bnc5=L.BatchNormalization(256),
            bnc6=L.BatchNormalization(256),
            bnc7=L.BatchNormalization(512),
            bnc8=L.BatchNormalization(512),

            bnd8=L.BatchNormalization(512),
            bnd7=L.BatchNormalization(256),
            bnd6=L.BatchNormalization(256),
            bnd5=L.BatchNormalization(128),
            bnd4=L.BatchNormalization(128),
            bnd3=L.BatchNormalization(64),
            bnd2=L.BatchNormalization(64),
            bnd1=L.BatchNormalization(32)
        )


    def forward(self, x):

        e0 = F.relu(self.bnc0(self.c0(x)))
        e1 = F.relu(self.bnc1(self.c1(e0)))
        e2 = F.relu(self.bnc2(self.c2(e1)))
        del e1
        e3 = F.relu(self.bnc3(self.c3(e2)))
        e4 = F.relu(self.bnc4(self.c4(e3)))
        del e3
        e5 = F.relu(self.bnc5(self.c5(e4)))
        e6 = F.relu(self.bnc6(self.c6(e5)))
        del e5
        e7 = F.relu(self.bnc7(self.c7(e6)))
        e8 = F.relu(self.bnc8(self.c8(e7)))

        d8 = F.relu(self.bnd8(self.dc8(F.concat([e7, e8]))))
        del e7, e8
        d7 = F.relu(self.bnd7(self.dc7(d8)))
        del d8
        d6 = F.relu(self.bnd6(self.dc6(F.concat([e6, d7]))))
        del d7, e6
        d5 = F.relu(self.bnd5(self.dc5(d6)))
        del d6
        d4 = F.relu(self.bnd4(self.dc4(F.concat([e4, d5]))))
        del d5, e4
        d3 = F.relu(self.bnd3(self.dc3(d4)))
        del d4
        d2 = F.relu(self.bnd2(self.dc2(F.concat([e2, d3]))))
        del d3, e2
        d1 = F.relu(self.bnd1(self.dc1(d2)))
        del d2
        d0 = self.dc0(F.concat([e0, d1]))

        return d0


    def __call__(self, x, t):

        h = self.forward(x)
        
        loss = F.softmax_cross_entropy(h, t, ignore_label=self.__ignore_label)
        accuracy = F.accuracy(h, t, ignore_label=self.__ignore_label)
        
        chainer.report({'loss': loss, 'accuracy': accuracy}, self)
        
        return loss

        
    @property
    def class_num(self):
        return self.__class_num
    

/usr/local/lib/python3.7/dist-packages/chainer/_environment_check.py:75: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy-cuda111) version 9.4.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy-cuda111>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  requirement=requirement, help=help))


In [5]:
import numpy as np
import cv2
import math

import chainer
import chainer.functions as F
from chainer import cuda, serializers, Variable


class SegmentationModel:

	def __init__(self, model_path, mean, gpu=0):

		# Load model
		self.__model = UNet()
		serializers.load_npz(model_path, self.__model)

		chainer.cuda.get_device(gpu).use()
		self.__model.to_gpu(gpu)

		# Add height and width dimensions to mean 
		self.__mean = mean[np.newaxis, np.newaxis, :]


	def apply_segmentation(self, image):

		image_in, crop = self.__preprocess(image)

		with chainer.using_config('train', False):
			score = self.__model.forward(image_in)
		
		score = F.softmax(score)
		score = cuda.to_cpu(score.data)[0]
		
		top, left, bottom, right = crop
		score = score[:, top:bottom, left:right]
		
		return score


	def apply_segmentation_to_mosaic(self, mosaic, grid_px=800, tile_overlap_px=200):

		h, w, _ = mosaic.shape

		assert ((grid_px + tile_overlap_px * 2) % 16 == 0), "(grid_px + tile_overlap_px * 2) must be divisible by 16"

		pad_y1 = tile_overlap_px
		pad_x1 = tile_overlap_px

		n_y = int(float(h) / float(grid_px))
		n_x = int(float(w) / float(grid_px))
		pad_y2 = n_y * grid_px + 2 * tile_overlap_px - h - pad_y1
		pad_x2 = n_x * grid_px + 2 * tile_overlap_px - h - pad_x1

		mosaic_padded = np.pad(mosaic, ((pad_y1, pad_y2), (pad_x1, pad_x2), (0, 0)), 'symmetric')

		H, W, _ = mosaic_padded.shape
		score_padded = np.zeros(shape=[self.__model.class_num, H, W], dtype=np.float32)

		for yi in range(n_y):
		    for xi in range(n_x):
		        
		        top = yi * grid_px
		        left = xi * grid_px
		        bottom = top + grid_px + 2 * tile_overlap_px
		        right = left + grid_px + 2 * tile_overlap_px
		        
		        tile = mosaic_padded[top:bottom, left:right]
		        
		        score_tile = self.apply_segmentation(tile)
		        
		        score_padded[:, top:bottom, left:right] = score_tile

		score = score_padded[:, pad_y1:-pad_y2, pad_x1:-pad_x2]

		return score


	def __preprocess(self, image):

		h, w, _ = image.shape
		h_padded = int(math.ceil(float(h) / 16.0) * 16)
		w_padded = int(math.ceil(float(w) / 16.0) * 16)

		pad_y1 = (h_padded - h) // 2
		pad_x1 = (w_padded - w) // 2
		pad_y2 = h_padded - h - pad_y1
		pad_x2 = w_padded - w - pad_x1

		image_padded = np.pad(image, ((pad_y1, pad_y2), (pad_x1, pad_x2), (0, 0)), 'symmetric')
		image_in = (image_padded - self.__mean) / 255.0
		image_in = image_in.transpose(2, 0, 1)
		image_in = image_in[np.newaxis, :, :, :]
		image_in = Variable(cuda.cupy.asarray(image_in, dtype=cuda.cupy.float32))

		top, left = pad_y1, pad_x1
		bottom, right = top + h, left + w

		return image_in, (top, left, bottom, right)


In [6]:
from os import path
from PIL import Image
import numpy as np
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt

In [7]:
# Load trained model
# Modify the the paths based on your trained model location if needed.
mean = np.load('/content/drive/MyDrive/4101-AI-Project/dataset/mean.npy')
model = SegmentationModel('/content/drive/MyDrive/4101-AI-Project/models/logs/model_iter_3035', mean)

In [8]:
# Load test-set
# Modify the the paths based on your data location if needed.
with open('/content/drive/MyDrive/4101-AI-Project/dataset/test.txt') as f:
    lines = f.readlines()

test_set = []
for line in lines:
    line = line.rstrip()
    test_set.append(line)

In [9]:
#  Evaluate model

# Modify the the paths based on your raster image and building label location.
image_dir = '/content/drive/MyDrive/4101-AI-Project/processedBuildingLabels/3band/3band'
label_dir = '/content/drive/MyDrive/4101-AI-Project/buildingMaskImages'

gt_labels = []
pred_labels = []

for test_data in tqdm(test_set):
    image_path = path.join(image_dir, test_data)
    label_path = path.join(label_dir, test_data)
    
    image = np.array(Image.open(image_path))
    label = np.array(Image.open(label_path))
    
    # Make gt_label, ground-truth building mask
    h, w = label.shape
    gt_label = np.zeros(shape=[h, w], dtype=np.int32) # 0: background
    gt_label[label > 0] = 1 # 1: "building"
    
    # Make pred_label, predicted building mask
    score = model.apply_segmentation(image)
    pred_label = np.argmax(score, axis=0)
    
    gt_labels.append(gt_label)
    pred_labels.append(pred_label)

100%|██████████| 1389/1389 [17:59<00:00,  1.29it/s]


In [10]:
result = eval_semantic_segmentation(pred_labels, gt_labels)
print(result)

{'iou': array([0.94615677, 0.50831679]), 'miou': 0.7272367773236843, 'pixel_accuracy': 0.9489958986034932, 'class_accuracy': array([0.9832502 , 0.59604563]), 'mean_class_accuracy': 0.7896479178727212}


In [11]:
building_class = 1
print("IoU for class Building = ", result['iou'][building_class])
print("Accuracy for class Building = ", result['class_accuracy'][building_class])

IoU for class Building =  0.5083167860973677
Accuracy for class Building =  0.5960456345838827


In [12]:
confusion = calc_semantic_segmentation_confusion(pred_labels, gt_labels)
print("confusion:")
print(confusion)

tn = confusion[0][0]
fp = confusion[0][1]
tp = confusion[1][1]
fn = confusion[1][0]

precision = float(tp) / float(tp + fp)
recall       = float(tp) / float(tp + fn)

print()
print("precision: ", precision)
print("recall: ", recall)

confusion:
[[221698815   3776669]
 [  8839613  13043089]]

precision:  0.7754623461288801
recall:  0.5960456345838827


# Evaluate Other Model

In [13]:
model = SegmentationModel('/content/drive/MyDrive/4101-AI-Project/models/logs/model_iter_2428', mean)

In [14]:

gt_labels = []
pred_labels = []

for test_data in tqdm(test_set):
    image_path = path.join(image_dir, test_data)
    label_path = path.join(label_dir, test_data)
    
    image = np.array(Image.open(image_path))
    label = np.array(Image.open(label_path))
    
    # Make gt_label, ground-truth building mask
    h, w = label.shape
    gt_label = np.zeros(shape=[h, w], dtype=np.int32) # 0: background
    gt_label[label > 0] = 1 # 1: "building"
    
    # Make pred_label, predicted building mask
    score = model.apply_segmentation(image)
    pred_label = np.argmax(score, axis=0)
    
    gt_labels.append(gt_label)
    pred_labels.append(pred_label)

100%|██████████| 1389/1389 [04:09<00:00,  5.57it/s]


In [15]:
result = eval_semantic_segmentation(pred_labels, gt_labels)
print(result)

{'iou': array([0.94734859, 0.5441041 ]), 'miou': 0.7457263445116342, 'pixel_accuracy': 0.9504615141380444, 'class_accuracy': array([0.97784374, 0.66831989]), 'mean_class_accuracy': 0.8230818161390682}


In [16]:
building_class = 1
print("IoU for class Building = ", result['iou'][building_class])
print("Accuracy for class Building = ", result['class_accuracy'][building_class])

IoU for class Building =  0.5441041029421586
Accuracy for class Building =  0.6683198902950833


In [17]:
confusion = calc_semantic_segmentation_confusion(pred_labels, gt_labels)
print("confusion:")
print(confusion)

tn = confusion[0][0]
fp = confusion[0][1]
tp = confusion[1][1]
fn = confusion[1][0]

precision = float(tp) / float(tp + fp)
recall       = float(tp) / float(tp + fn)

print()
print("precision: ", precision)
print("recall: ", recall)

confusion:
[[220479791   4995693]
 [  7258057  14624645]]

precision:  0.745381909322867
recall:  0.6683198902950833
